# Extraction of relevant terms and parsing of list attributes

In [1]:
#TODO: remove after development 
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 5400)
pd.set_option('display.max_rows', 300)
pd.set_option('max_colwidth', 60)

## Loading data

In [4]:
import pickle

#seed_name = 'hair_dryer'
#seed_name = 'video_codec'
seed_name = 'diesel'
#seed_name = "contact_lens"
#seed_name = "3d_printer"

src_dir = "."

with open(src_dir + "/data/" + seed_name + '/cleaned_text.pkl', 'rb') as infile:
    df = pickle.load(infile)

In [5]:
from unidecode import unidecode

df['raw_assignees'] = df['assignees_harmonized'].map(lambda r: unidecode(r).split(','))

print(df.shape)
df.head()

(4482, 17)


,family_id,title_text,abstract_text,claims_text,legacy_terms,priority_date,country_code,pub_num,publication_number,assignees_harmonized,ipcs,refs,cpcs,ExpansionLevel,text_prep,tokens,raw_assignees
0,2018095213,ALUMINUM PLATE AND COOLER HAVING THE SAME,An aluminum plate and an EGR cooler may include a cooler...,"1. A cooler apparatus, comprising:, a housing in which a...","exhaust gas recirculating, exhaust gas flow, aluminum pl...",20170728,EP,3435021,EP-3435021,"Hyundai Motor Company,Kia Motors Corporation","F28F01/08,F28D07/16,F28D01/00,F28F01/08,F28D07/16,F28D01/00",,,Seed,ALUMINUM PLATE AND COOLER HAVING THE SAME An aluminum pl...,"[aluminum, plate, cooler, aluminum, plate, egr, cooler, ...","[Hyundai Motor Company, Kia Motors Corporation]"
1,2018091406,FLUID SPRAY INJECTORS,A fluid spray nozzle tip (100;200) can include a feed ho...,"1. A fluid spray nozzle tip, comprising:, a feed hole bo...","fluid spray, feed hole body, pintle sealing surface body...",20170719,EP,3431730,EP-3431730,Delavan Inc.,"F01N03/20,F02M01/16,F02M01/18,F01N03/20,F02M01/16,F02M01/18",,,Seed,FLUID SPRAY INJECTORS A fluid spray nozzle tip (100;200)...,"[fluid, spray, injectors, fluid, spray, nozzle, tip, fee...",[Delavan Inc.]
2,2018087099,POLLUTANT ABATEMENT SYSTEM FOR AN INTERNAL COMBUSTION EN...,A system for the abatement of the pollutants produced by...,1. A system for the abatement of the pollutants produced...,"pollutant abatement system, treatment portion, ats porti...",20170713,EP,3428440,EP-3428440,FPT Industrial S.p.A.,"F02M06/06,F02M06/14,F01N03/20,F01N03/08,F01N09/00,F01N03...",,,Seed,POLLUTANT ABATEMENT SYSTEM FOR AN INTERNAL COMBUSTION EN...,"[pollutant, abatement, system, internal, combustion, eng...",[FPT Industrial S.p.A.]
3,2018087098,POLLUTANT ABATEMENT SYSTEM OF AN INTERNAL COMBUSTION ENG...,An abatement system of pollutants produced by an interna...,1. An abatement system of pollutants produced by an inte...,"treatment portion, combustion engine, exhaust gas, bypas...",20170713,EP,3428415,EP-3428415,FPT Industrial S.p.A.,"F01N03/08,F01N03/20,F02M06/00,F02M06/06,F02M06/15,F02M06...",,,Seed,POLLUTANT ABATEMENT SYSTEM OF AN INTERNAL COMBUSTION ENG...,"[pollutant, abatement, system, internal, combustion, eng...",[FPT Industrial S.p.A.]
5,2018082871,METHOD AND SYSTEM FOR THE PURIFICATION OF EXHAUST GAS FR...,The invention discloses a method for the purification of...,1. Method for the purification of exhaust gas from an in...,"combustion engine, exhaust gas, scr filter, particulate ...",20120427,EP,3425182,EP-3425182,UMICORE AG & CO. KG,"F01N03/20,F01N03/035,B01J09/00,B01D03/94,B01J07/02,B01J0...",,,Seed,METHOD AND SYSTEM FOR THE PURIFICATION OF EXHAUST GAS FR...,"[system, purification, exhaust, gas, internal, combustio...",[UMICORE AG & CO. KG]


## Extracting useful attributes

In [6]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def dedupe_fixed(contains_dupes, threshold=70, scorer=fuzz.token_set_ratio):
    """This convenience function takes a list of strings containing duplicates and uses fuzzy matching to identify
    and remove duplicates. Specifically, it uses the process.extract to identify duplicates that
    score greater than a user defined threshold. Then, it looks for the longest item in the duplicate list
    since we assume this item contains the most entity information and returns that. It breaks string
    length ties on an alphabetical sort.
    Note: as the threshold DECREASES the number of duplicates that are found INCREASES. This means that the
        returned deduplicated list will likely be shorter. Raise the threshold for fuzzy_dedupe to be less
        sensitive.
    Args:
        contains_dupes: A list of strings that we would like to dedupe.
        threshold: the numerical value (0,100) point at which we expect to find duplicates.
            Defaults to 70 out of 100
        scorer: Optional function for scoring matches between the query and
            an individual processed choice. This should be a function
            of the form f(query, choice) -> int.
            By default, fuzz.token_set_ratio() is used and expects both query and
            choice to be strings.
    Returns:
        A deduplicated list. For example:
            In: contains_dupes = ['Frodo Baggin', 'Frodo Baggins', 'F. Baggins', 'Samwise G.', 'Gandalf', 'Bilbo Baggins']
            In: fuzzy_dedupe(contains_dupes)
            Out: ['Frodo Baggins', 'Samwise G.', 'Bilbo Baggins', 'Gandalf']
        """

    extractor = []
    processed = []

    # iterate over items in *contains_dupes*
    for item in contains_dupes:
        if item not in processed and item not in extractor:
            # return all duplicate matches found
            matches = process.extract(item, contains_dupes, limit=None, scorer=scorer)
            # filter matches based on the threshold
            filtered = [x for x in matches if x[1] >= threshold]
            # if there is only 1 item in *filtered*, no duplicates were found so append to *extracted*
            if len(filtered) == 1:
                extractor.append(filtered[0][0])

            else:
                # alpha sort
                filtered = sorted(filtered, key=lambda x: x[0])
                # length sort
                filter_sort = sorted(filtered, key=lambda x: len(x[0]))
                #filter_sort = sorted(filtered, key=lambda x: len(x[0]), reverse=True)
                # take first item as our 'canonical example'
                extractor.append(filter_sort[0][0])
                
            processed.append([i[0] for i in filtered])

    # uniquify *extractor* list
    
    set_extractor = set(extractor)
    extractor = list(set_extractor)

    # check that extractor differs from contain_dupes (e.g. duplicates were found)
    # if not, then return the original list
    if len(extractor) == len(contains_dupes):
        return contains_dupes
    else:
        return extractor

In [7]:
assignees = [item for sublist in df['raw_assignees'].values for item in sublist]
assignees_set = set(assignees)
print(len(assignees_set))
#assignees_set

1118


In [13]:
threshold = 88 #hair dryer


#scorer = fuzz.token_sort_ratio
scorer = fuzz.token_set_ratio

deduplicated = sorted(list(dedupe_fixed(assignees_set, threshold=threshold, scorer=scorer)))

print(len(deduplicated))

700


In [14]:
deduplicated_dictionary = {}
for d in deduplicated:
    matches = process.extractBests(d, assignees_set, score_cutoff=threshold, scorer=scorer)
    deduplicated_dictionary[d] = matches
    
for k, v in deduplicated_dictionary.items():
    if len(v) > 1:
        print(k, v)

AB VOLVO [('AB VOLVO', 100), ('Volvo Construction Equipment AB', 100), ('Volvo Construction Equipment Holding Sweden AB', 100), ('VOLVO LASTVAGNAR AB', 100), ('Volvo Lastvagnar AB', 100)]
ACS Industries Inc. [('Acs Industries Inc.', 100), ('ACS Industries Inc.', 100), ('ARVIN INDUSTRIES INC.', 88), ('SUMITOMO ELECTRIC INDUSTRIES INC.', 88)]
AKZO N.V. [('Akzo Nobel N.V.', 100), ('AKZO N.V.', 100)]
ARCO Chemical Technology L.P. [('ARCO Chemical Technology L.P.', 100), ('Lyondell Chemical Technology L.P.', 90)]
ASAHI GLASS COMPANY LTD. [('ASAHI GLASS COMPANY LTD.', 100), ('Asahi Glass Company Limited', 92), ('Asahi Glass Co. Ltd.', 91)]
ASEC Manufacturing Company [('ASEC Manufacturing Company', 100), ('Watlow Electric Manufacturing Company', 89), ('MINNESOTA MINING AND MANUFACTURING COMPANY', 89)]
AUDI AG [('AUDI AG', 100), ('Audi AG', 100)]
Alantum [('Alantum', 100), ('Alantum GmbH & Co. KG', 100), ('Alantum Corporation', 100)]
Aristotle University Thessaloniki [('ARISTOTLE UNIVERSITY OF

In [12]:
from disamby import Disamby
import disamby.preprocessors as pre

assignees_df = pd.DataFrame(data={'assignee': list(assignees_set)})

pipeline = [pre.normalize_whitespace, pre.compact_abbreviations, pre.remove_punctuation, lambda x: pre.trigram(x) + pre.split_words(x)]
dis = Disamby(assignees_df, pipeline)
disambiguated = dis.disambiguated_sets(threshold=0.5, verbose=False)

print(len(disambiguated))

for s in disambiguated:
    if len(s) > 1:
        print(assignees_df.loc[s].values)

838
[['ERNST-APPARATEBAU GmbH & Co.']
 ['ERNST APPARATEBAU GmbH & Co.']]
[['Hengst GmbH & Co. KG']
 ['Hengst SE & Co. KG']]
[['Alantum Corporation']
 ['Alantum GmbH & Co. KG']
 ['Alantum']]
[['Iseki & Co. Ltd.']
 ['ISEKI & CO. LTD.']]
[['SA 1/4 d-Chemie AG']
 ['SA 1/4 d-Chemie IP GmbH & Co. KG']]
[['Behr GmbH & Co.']
 ['Behr GmbH & Co. KG']]
[['Schmidt + Clemens GmbH & Co. KG']
 ['Schmidt + Clemens GmbH + Co. KG']]
[['Mitsui Mining and Smelting Co. Ltd']
 ['Mitsui Mining & Smelting Co. Ltd']
 ['MITSUI MINING & SMELTING CO. LTD.']
 ['Mitsui Mining and Smelting Co. Ltd.']]
[['UMICORE AG & CO. KG']
 ['Umicore AG & Co. KG']]
[['Pirelli & C. Ambiente S.r.l.']
 ['Pirelli & C. Eco Technology S.p.A.']]
[['DEERE & COMPANY']
 ['Deere & Company']]
[['Heesung Engelhard Corporation']
 ['Heesung Catalysts Corporation']]
[['Mann + Hummel GmbH']
 ['MANN + HUMMEL GMBH']]
[['PetrA3leo Brasileiro S.A. Petrobras']
 ['Petroleo Brasileiro S.A. Petrobras']
 ['Petroleo Brasileiro S.A. - PETROBRAS']]
[['J. Ebe

In [11]:
def disambiguate_assignees(assignee_list):
    result = []
    for assignee in assignee_list:
        match = process.extractOne(assignee, deduplicated)
        result.append(match[0])
    return list(set(result))

#df['disambiguated_assignees'] = df['raw_assignees'].map(lambda x: disambiguate_assignees(x))
df.head()

,family_id,title_text,abstract_text,claims_text,legacy_terms,priority_date,country_code,pub_num,publication_number,assignees_harmonized,ipcs,refs,cpcs,ExpansionLevel,text_prep,tokens,raw_assignees
0,2018095213,ALUMINUM PLATE AND COOLER HAVING THE SAME,An aluminum plate and an EGR cooler may include a cooler...,"1. A cooler apparatus, comprising:, a housing in which a...","exhaust gas recirculating, exhaust gas flow, aluminum pl...",20170728,EP,3435021,EP-3435021,"Hyundai Motor Company,Kia Motors Corporation","F28F01/08,F28D07/16,F28D01/00,F28F01/08,F28D07/16,F28D01/00",,,Seed,ALUMINUM PLATE AND COOLER HAVING THE SAME An aluminum pl...,"[aluminum, plate, cooler, aluminum, plate, egr, cooler, ...","[Hyundai Motor Company, Kia Motors Corporation]"
1,2018091406,FLUID SPRAY INJECTORS,A fluid spray nozzle tip (100;200) can include a feed ho...,"1. A fluid spray nozzle tip, comprising:, a feed hole bo...","fluid spray, feed hole body, pintle sealing surface body...",20170719,EP,3431730,EP-3431730,Delavan Inc.,"F01N03/20,F02M01/16,F02M01/18,F01N03/20,F02M01/16,F02M01/18",,,Seed,FLUID SPRAY INJECTORS A fluid spray nozzle tip (100;200)...,"[fluid, spray, injectors, fluid, spray, nozzle, tip, fee...",[Delavan Inc.]
2,2018087099,POLLUTANT ABATEMENT SYSTEM FOR AN INTERNAL COMBUSTION EN...,A system for the abatement of the pollutants produced by...,1. A system for the abatement of the pollutants produced...,"pollutant abatement system, treatment portion, ats porti...",20170713,EP,3428440,EP-3428440,FPT Industrial S.p.A.,"F02M06/06,F02M06/14,F01N03/20,F01N03/08,F01N09/00,F01N03...",,,Seed,POLLUTANT ABATEMENT SYSTEM FOR AN INTERNAL COMBUSTION EN...,"[pollutant, abatement, system, internal, combustion, eng...",[FPT Industrial S.p.A.]
3,2018087098,POLLUTANT ABATEMENT SYSTEM OF AN INTERNAL COMBUSTION ENG...,An abatement system of pollutants produced by an interna...,1. An abatement system of pollutants produced by an inte...,"treatment portion, combustion engine, exhaust gas, bypas...",20170713,EP,3428415,EP-3428415,FPT Industrial S.p.A.,"F01N03/08,F01N03/20,F02M06/00,F02M06/06,F02M06/15,F02M06...",,,Seed,POLLUTANT ABATEMENT SYSTEM OF AN INTERNAL COMBUSTION ENG...,"[pollutant, abatement, system, internal, combustion, eng...",[FPT Industrial S.p.A.]
5,2018082871,METHOD AND SYSTEM FOR THE PURIFICATION OF EXHAUST GAS FR...,The invention discloses a method for the purification of...,1. Method for the purification of exhaust gas from an in...,"combustion engine, exhaust gas, scr filter, particulate ...",20120427,EP,3425182,EP-3425182,UMICORE AG & CO. KG,"F01N03/20,F01N03/035,B01J09/00,B01D03/94,B01J07/02,B01J0...",,,Seed,METHOD AND SYSTEM FOR THE PURIFICATION OF EXHAUST GAS FR...,"[system, purification, exhaust, gas, internal, combustio...",[UMICORE AG & CO. KG]
